In [ ]:
%pylab inline
style.use('http://johannesfeist.eu/misc/jf.mplstyle')
np.set_printoptions(linewidth=200)

In [ ]:
from qutip import *

In [ ]:
from jftools.short_iterative_lanczos import lanczos_timeprop

In [ ]:
from scipy.sparse import lil_matrix
N = 20000
H = lil_matrix((N,N))
H[range(N),range(N)] = -2.
H[range(N-1),range(1,N)] = 1
H[range(1,N),range(N-1)] = 1
H = H.tocsr()

In [ ]:
phi0 = exp(-(arange(N)-15000)**2/(2*300**2)-1.5j*arange(N)) + exp(-(arange(N)-5000)**2/(2*50**2)+1j*arange(N))
phi0 /= norm(phi0)
phir = randn(N).astype(complex)
phir /= norm(phir)

In [ ]:
from scipy.sparse._sparsetools import csr_matvec
def Hv(t,phi,Hphi):
    M,N = H.shape
    Hphi.fill(0)
    csr_matvec(M,N,H.indptr,H.indices,H.data,phi,Hphi)
    return Hphi

In [ ]:
def Hv2(t,phi,Hphi):
    return H.dot(phi).view(type(phi))
def Hv3(t,phi,Hphi):
    Hphi[:] = H.dot(phi)
    return Hphi

In [ ]:
a = H.dot(phi0)
b = empty_like(a); Hv(0,phi0,b)
c = Hv2(0,phi0,None)
d = empty_like(a); Hv3(0,phi0,d)
print(np.all(a==b))
print(np.all(a==c))
print(np.all(a==d))
%timeit a = H.dot(phi0)
%timeit Hv(0,phi0,b)
%timeit c = Hv2(0,phi0,None)
%timeit Hv3(0,phi0,d)

In [ ]:
Hq = Qobj(H)
phi0q = Qobj(phi0)
phirq = Qobj(phir)

In [ ]:
prop = lanczos_timeprop(H,12,1e-10)
resa = prop.propagate(phir,[0,.5])

In [ ]:
prop = lanczos_timeprop(Hv,12,1e-10)
resb = prop.propagate(phir,[0,.5])
prop = lanczos_timeprop(Hv2,12,1e-10)
resc = prop.propagate(phir,[0,.5])
prop = lanczos_timeprop(Hv3,12,1e-10)
resd = prop.propagate(phir,[0,.5])

In [ ]:
prop = lanczos_timeprop(Hq,12,1e-10)
resq = prop.propagate(phirq,[0,.5])

In [ ]:
print(np.all(resa[1] == resq[1].full().squeeze()),
      np.all(resa[1] == resb[1]),
      np.all(resa[1] == resc[1]),
      np.all(resa[1] == resd[1]))

In [ ]:
for Hf in H,Hv,Hv2,Hv3:
    prop = lanczos_timeprop(Hf,12,1e-10)
    %timeit prop.propagate(phir,[0,.5])

In [ ]:
from scipy.sparse.linalg import expm_multiply
dt = 0.5
prop = lanczos_timeprop(H,12,1e-12)
phi1 = prop.propagate(phi0,[0,dt])[-1]
phi2 = expm_multiply(-1j*dt*H,phi0)
np.allclose(phi1,phi2)

In [ ]:
%timeit lanczos_timeprop(H,12,1e-12).propagate(phi0,[0.,dt])
prop = lanczos_timeprop(H,12,1e-12)
%timeit prop.propagate(phi0,[0.,dt])
prop.phia[0][:] = phi0
%timeit prop._step(0,dt)
%timeit phi2 = expm_multiply(-1j*dt*H,phi0)
#%timeit sesolve(Hq,phi0q,[0.,dt],[])

In [ ]:
ts = linspace(0,100,11)
prop = lanczos_timeprop(H,12,1e-12)
for p0 in phi0,phir:
    p0q = Qobj(p0)
    %time phiLs = prop.propagate(p0,ts)
    %time resQ = sesolve(Hq,p0q,ts,[],options=Options(atol=1e-10,rtol=1e-10))
    phiLs = array(phiLs)
    phiQs = array([s.full() for s in resQ.states]).squeeze()
    print(norm(phiLs), norm(phiQs-phiLs), norm(phiQs[-1]-phiLs[-1]))
    f,axs = subplots(1,2,figsize=(8,4),sharex=True,sharey=True)
    axs[0].pcolormesh(ts,arange(N),abs(phiQs.T)**2)
    axs[1].pcolormesh(ts,arange(N),abs(phiLs.T)**2)
    axs[0].autoscale(tight=True)
    f.tight_layout(pad=0.5)

In [ ]:
convgs = [1e-6,1e-8,1e-10,1e-12,1e-14,1e-16]
refphi = lanczos_timeprop(H,12,1e-14).propagate(phir,ts)[-1]

qsolve = lambda tol: array([s.full() for s in sesolve(Hq,phirq,ts,[],options=Options(atol=tol,rtol=tol)).states]).squeeze()
phiLs_convg, phiQs_convg = {}, {}
for c in convgs:
    if c>=1e-14:
        print('SIL',c,end=' ')
        tt = %timeit -qo -n1 -r1 phiLs_convg[c] = lanczos_timeprop(H,12,c).propagate(phir,ts)
        phif = phiLs_convg[c][-1]
        print("%.4f %.4e"%(tt.best,mean(abs(refphi-phif)/abs(refphi))))
    print('RK ',c,end=' ')
    tt = %timeit -qo -n1 -r1 phiQs_convg[c] = qsolve(c)
    phif = phiQs_convg[c][-1]
    print("%.4f %.4e"%(tt.best,mean(abs(refphi-phif)/abs(refphi))))

In [ ]:
def err(a,b,ea,eb):
    dd = dict(Q=phiQs_convg,L=phiLs_convg)
    phia = dd[a][ea][-1]
    phib = dd[b][eb][-1]
    return 2*abs(phia-phib)/(abs(phia)+abs(phib))
def ploterr(a,b,ea,eb):
    relerr = err(a,b,ea,eb)
    names = dict(Q='RK',L='SIL')
    plot(relerr,label="%s %g vs %s %g"%(names[a],ea,names[b],eb))
ploterr('Q','Q',1e-16,1e-06)
ploterr('L','L',1e-14,1e-06)
ploterr('L','L',1e-14,1e-12)
ploterr('Q','L',1e-16,1e-14)
ploterr('L','L',1e-14,1e-10)
ploterr('L','L',1e-14,1e-8)
yscale('log')
ylabel('rel. error')
legend(frameon=True,fontsize=14);

In [ ]:
meanerrsLL = array([(e,mean(err('L','L',1e-14,e))) for e in [1e-6,1e-8,1e-10,1e-12]]).T
meanerrsQL = array([(e,mean(err('Q','L',1e-16,e))) for e in [1e-6,1e-8,1e-10,1e-12,1e-14]]).T
meanerrsLQ = array([(e,mean(err('L','Q',1e-14,e))) for e in [1e-6,1e-8,1e-10,1e-12,1e-14,1e-16]]).T
meanerrsQQ = array([(e,mean(err('Q','Q',1e-16,e))) for e in [1e-6,1e-8,1e-10,1e-12,1e-14]]).T
plot(meanerrsLL[0],meanerrsLL[1],'o-',label='LL')
plot(meanerrsQQ[0],meanerrsQQ[1],'o-',label='QQ')
plot(meanerrsQL[0],meanerrsQL[1],'o--',label='QL')
plot(meanerrsLQ[0],meanerrsLQ[1],'o--',label='LQ')
legend()
yscale('log')
xscale('log')

In [ ]:
class testclass:
    def __init__(self,data):
        self.data = asarray(data,dtype=complex)
    def norm(self):
        return norm(self.data)
    def dot(self,other):
        return vdot(self.data,other.data)
    def copy(self):
        return testclass(self.data.copy())
    def __mul__(self,a):
        return testclass(a*self.data)
    def __rmul__(self,a):
        return self*a
    def __imul__(self,a):
        self.data *= a
        return self
    def __isub__(self,other):
        self.data -= other.data
        return self
    def __iadd__(self,other):
        self.data += other.data
        return self

N = 501
x,dx = linspace(-10,10,N,retstep=True)
H = lil_matrix((N,N))
H[range(N),range(N)] = 1./dx**2 + 0.5*x**2
H[range(N-1),range(1,N)] = -0.5/dx**2
H[range(1,N),range(N-1)] = -0.5/dx**2
H = H.tocsr()
D = lil_matrix(diag(x)).tocsr()

In [ ]:
vals,vecs = eigh(H.toarray())
plot(vals[:20],'o')

In [ ]:
for ii in range(10):
    plot(x,0*x+vals[ii],'0.5',lw=0.5)
    plot(x,3*vecs[:,ii]+vals[ii])
autoscale(axis='x',tight=True)
autoscale(False)
plot(x,0.5*x**2,'k')

In [ ]:
omega = 0.2
sigma = 2*pi/omega
EF = lambda t: 2.*exp(-t**2/(2*sigma**2))*sin(omega*t)

def testHfun(t,phi,Hphi):
    Hphi.data[:] = H.dot(phi.data)
    Hphi.data += EF(t)*D.dot(phi.data)
    return Hphi

In [ ]:
ts = linspace(-4*sigma,4*sigma,201)
plot(ts,EF(ts))

In [ ]:
phi0 = testclass(vecs[:,0])
prop = lanczos_timeprop(testHfun,12,1e-12)
%time res = prop.propagate(phi0,ts,maxHT=2*pi/omega / 40)

In [ ]:
%time res2 = prop.propagate(phi0,ts,maxHT=2*pi/omega / 80)

In [ ]:
QHD = [Qobj(H),[Qobj(D),lambda t,args: EF(t)]]
%time resq = sesolve(QHD,Qobj(vecs[:,0]),ts,[],options=Options(nsteps=10000,rtol=1e-8))

In [ ]:
QHD = [Qobj(H),[Qobj(D),'2.*exp(-t**2/(2*{sigma}**2))*sin({omega}*t)'.format(**globals())]]
%time resq2 = sesolve(QHD,Qobj(vecs[:,0]),ts,[],options=Options(nsteps=10000,rtol=1e-8))

In [ ]:
QHD = [Qobj(H),[Qobj(D),'2.*exp(-t**2/(2*{sigma}**2))*sin({omega}*t)'.format(**globals())]]
%time resq3 = sesolve(QHD,Qobj(vecs[:,0]),ts,[],options=Options(nsteps=10000,rtol=1e-14,atol=1e-14))

In [ ]:
phisL = array([r.data for r in res])
phisL2 = array([r.data for r in res2])
phisQ = array([r.full() for r in resq.states]).squeeze()
phisQ2 = array([r.full() for r in resq2.states]).squeeze()
phisQ3 = array([r.full() for r in resq3.states]).squeeze()

In [ ]:
print(np.allclose(phisL,phisL2,rtol=1e-4,atol=1e-4))
print(np.allclose(phisL2,phisQ3,rtol=1e-3,atol=1e-3))
print(np.allclose(phisL,phisQ ,rtol=1e-3,atol=1e-4))
print(np.allclose(phisL,phisQ2,rtol=1e-3,atol=1e-4))
print(np.allclose(phisL,phisQ3,rtol=1e-3,atol=1e-3))

In [ ]:
f,axs = subplots(1,4,figsize=(13,5),sharex=True,sharey=True)
for ax,p,lab in zip(axs,
                    [phisL2,phisQ3,phisL2-phisQ3,phisQ-phisQ3],
                    ['SIL',  'RK',    'SIL-RK',  'RK py-cy']):
    p = abs(p).T
    im = ax.pcolormesh(ts,x,p,shading='gouraud')
    ax.text(0.02,0.02,'Max=%.2e'%p.max(),transform=ax.transAxes,color='w')
    ax.text(0.5,0.98,lab,transform=ax.transAxes,color='w',ha='center',va='top')
axs[0].autoscale(tight=True)
f.tight_layout(pad=0.01)
cb = colorbar(im,ax=list(axs),orientation='horizontal',shrink=0.8,pad=0.1,aspect=40)
cb.set_ticks([0,p.max()])
cb.set_ticklabels(['0','max'])